# Extracción de petróleo OilyGiant.

## Introducción 
La empresa OilyGiant planea invertir $100,000,000 USD en el desarrollo de 200 pozos petroleros en tres regiones seleccionadas con alto potencial para la extracción de petróleo. De cada región se han identificado 500 puntos de interés, y el objetivo es reducir esta selección a solo una región y los 200 mejores puntos para la extracción.

Para lograrlo, se entrenará un modelo de machine learning utilizando regresión lineal para analizar la viabilidad de cada región, evaluando el beneficio bruto esperado. Para validar la fiabilidad del modelo, se aplicará la técnica de bootstrapping, a partir de la cual se calculará el beneficio promedio de cada región y se analizará el intervalo de confianza del 95%. Además, se realizarán cálculos para evaluar los riesgos de pérdidas, asegurando que se mantengan solo las regiones con un riesgo inferior al 2.5%.

## Cargar los datos

In [17]:
# Importar librerias
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
# Decargar los dataset
data_0 = pd.read_csv('../dataset/geo_data_0.csv')
data_1 = pd.read_csv('../dataset/geo_data_1.csv')
data_2 = pd.read_csv('../dataset/geo_data_2.csv')

### Procesamiento de datos

In [19]:
# Muestra la estructura de los datos de data_0
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [20]:
# Vista general de los datos
data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [21]:
# Muestra la estructura de los datos de data_1
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [22]:
# Vista general de los datos
data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [23]:
# Muestra la estructura de los datos de data_2
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [24]:
# Vista general de los datos
data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [25]:
# Revisar los 3 dataset en busca de duplicados, si el resultado es distinto de 0 se revisaran de forma indvidual
data_0.duplicated().sum()+data_1.duplicated().sum()+data_2.duplicated().sum()

0

### Descripción de datos
Los 3 DataFrame presentan la misma estructura de datos, no contienen duplicados ni datos nulos, a continuacion se definen las columnas de las tablas `geo_data`:
- `id`: Identificador único de pozo de petróleo
- `f0`, `f1`, `f2`: Tres características de los puntos (su significado específico no es importante, pero las características en sí son significativas).
- `product`: Volumen de reservas en el pozo de petróleo (miles de barriles).

## Entrenamiento del Modelo

In [26]:
# Función para entrenar y evaluar el modelo
def train_and_evaluate(data):
    # Características y objetivo
    X = data[['f0', 'f1', 'f2']]
    y = data['product']
    
    # Dividir en conjunto de entrenamiento y validación (75:25)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=1234)
    
    # Entrenar el modelo de regresión lineal
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Hacer predicciones en el conjunto de validación
    predictions = model.predict(X_val)
    
    # Guardar las predicciones y las respuestas correctas
    results = pd.DataFrame({'Predictions': predictions, 'Actual': y_val})
    
    # Calcular volumen medio de reservas predicho
    mean_predicted_reserves = predictions.mean()
    
    # Calcular el RMSE (raíz del error cuadrático medio)
    rmse = mean_squared_error(y_val, predictions, squared=False)
    
    print(f"Volumen medio predicho: {mean_predicted_reserves}")
    print(f"RMSE: {rmse}")
    
    return results

# Entrenar y evaluar el modelo en cada conjunto de datos
print("Región 0:")
results_0 = train_and_evaluate(data_0)
print("Región 1:")
results_1 = train_and_evaluate(data_1)
print("Región 2:")
results_2 = train_and_evaluate(data_2)

Región 0:
Volumen medio predicho: 92.43045952558441
RMSE: 37.562394183637785
Región 1:
Volumen medio predicho: 68.80248249798368
RMSE: 0.8939344738117714
Región 2:
Volumen medio predicho: 94.91573817913459
RMSE: 40.10318099258418


### Observaciones
En las regiones 0 y 2, los volúmenes predichos son más altos (alrededor de 92.43 y 94.92, respectivamente) en comparación con la región 1 (68.80). Esto sugiere que las regiones 0 y 2 podrían tener mayores reservas de petróleo, basándonos solo en el volumen predicho.

El RMSE mide el error de las predicciones: cuanto mayor sea el RMSE, más variable es la predicción respecto a los valores reales. Para las regiones 0 y 2, el RMSE es bastante alto (alrededor de 37.56 y 40.10), lo que significa que las predicciones en esas regiones son más imprecisas. En cambio, el RMSE para la región 1 es mucho más bajo (0.89), lo que indica que el modelo es mucho más preciso para esa región.

## Calcular la Viabilidad
En el siguiente paso, los datos se ordenan de mayor a menor volumen para seleccionar los 200 pozos con mayor cantidad de petróleo en cada región. Posteriormente, se calcula el volumen promedio de estos pozos para evaluar la rentabilidad estimada por pozo en cada región en comparación con el umbral de rentabilidad.

In [27]:
# Almacena en variables las constantes y valores necesarios
total_investment = 100000000 
oil_wells = 200
revenue_per_unit = 4500
investment_per_well = total_investment / revenue_per_unit / oil_wells # umbral de rentabilidad

def reserve_difference(results):
    # Seleccionar y almacenar los 200 pozos con el volumen más alto 
    top_volumes = results['Predictions'].sort_values(ascending = False).head(oil_wells)
    
    # Calcular el promedio de los pozos seleccionados y restar el umbral de rentabilidad
    difference  = top_volumes.mean()-investment_per_well
    
    return difference  
# Presentar resultados de las comparaciones con el umbral
print("Diferencia entre el umbral y la region 0: {:.2f}".format(reserve_difference(results_0)))
print("Diferencia entre el umbral y la region 1: {:.2f}".format(reserve_difference(results_1)))
print("Diferencia entre el umbral y la region 2: {:.2f}".format(reserve_difference(results_2)))

Diferencia entre el umbral y la region 0: 43.13
Diferencia entre el umbral y la region 1: 27.58
Diferencia entre el umbral y la region 2: 38.87


### Conclusion
Al igual que en el análisis anterior (la evaluacion del modelo), se observa una marcada diferencia en la Región 1, cuyo volumen promedio queda 26.83 unidades por encima del umbral de rentabilidad, en comparación con las Regiones 0 y 2, que muestran diferencias de 43.13 y 38.87 unidades, respectivamente, sobre el mismo umbral. Esto sugiere que las Regiones 0 y 2 superan la rentabilidad esperada con un margen significativamente mayor que la Región 1.

## Beneficio Bruto
Para calcular el beneficio bruto, primero seleccionaremos los 200 pozos de mayor volumen en cada región y sumaremos sus reservas. Luego, multiplicaremos este valor total por 4500 USD, que es el ingreso esperado por cada unidad de producción en miles de barriles. Esto nos dará el ingreso bruto para cada región. Finalmente, restaremos la inversión total de 100 millones USD para obtener el beneficio neto esperado en cada región.

In [28]:
def profit(results):
    # Almacenar los 200 pozos con el volumen más alto (volumen predicho)
    top_predictions=results['Predictions'].sort_values(ascending = False).head(oil_wells)
    
    # Calcular los ingresos multiplicando las reservas totales por el valor por unidad
    income  = top_predictions.sum() * revenue_per_unit
    
    # Restar la inversión total para obtener la ganancia bruta
    gross_profit   = income - total_investment
    
    return gross_profit , top_predictions
# Llamada a la función y almacenamiento de los resultados
profit_region_0, top_predictions_0 = profit(results_0)
profit_region_1, top_predictions_1 = profit(results_1)
profit_region_2, top_predictions_2 = profit(results_2)
# Presentar resultados de las comparaciones con el umbral, usando separador de miles y formato de moneda
print("Beneficio bruto región 0: ${:,.2f}".format(profit_region_0))
print("Beneficio bruto región 1: ${:,.2f}".format(profit_region_1))
print("Beneficio bruto región 2: ${:,.2f}".format(profit_region_2))

Beneficio bruto región 0: $38,821,157.44
Beneficio bruto región 1: $24,824,861.69
Beneficio bruto región 2: $34,983,487.05


Los resultados del cálculo de beneficio bruto muestran una diferencia importante entre las tres regiones. La región 1 genera un beneficio bruto significativamente menor, con `$24,824,861.69`, en comparación con las regiones 0 y 2, lo que concuerda con los análisis previos. La región 0 supera a la región 2 en términos de ganancia bruta, con `$38,821,157.44` y `$34,983,487.05` respectivamente.

La región 0 destaca consistentemente con una rentabilidad proyectada superior en comparación con las regiones 1 y 2. Aunque la diferencia entre las regiones 0 y 2 no es tan significativa, ambas se consideran altamente rentables en comparación con la región 1, que muestra un rendimiento considerablemente menor. Dado que la región 2 ha demostrado un volumen promedio de extracción superior en la validación del modelo, esta región sigue siendo una alternativa viable, aunque la región 0 se considera la opción preferida para la inversión .

## Calcula riesgos y ganancias para cada región:

### Bootstraping
La funcion a continuacion, transforma una Serie de predicciones en un DataFrame para su posterior uso.

Parámetros:
predictions (pd.Series): Serie de valores de predicción.

Retorna:
pd.DataFrame: DataFrame con los valores de predicción.
    

In [29]:
# Convierte una serie de predicciones a un DataFrame para su uso en el análisis de bootstrap.
def mk_df(predictions):
    predictions_df = pd.DataFrame(predictions)
    return predictions_df

# Transformación de predicciones a DataFrames para cada región.
predict_rg0_df = mk_df(top_predictions_0)
predict_rg1_df = mk_df(top_predictions_1)
predict_rg2_df = mk_df(top_predictions_2)

Esta función genera la distribución de los beneficios mediante la técnica de bootstrapping.

Parámetros:
predictions (pd.DataFrame): DataFrame de predicciones de los pozos seleccionados en una región.
n_samples (int): Número de muestras bootstrap a generar (por defecto es 1000).

Retorna:
pd.Series: Serie que contiene los beneficios netos calculados para cada muestra bootstrap.

In [30]:
# Realiza bootstrapping sobre los datos de predicción de pozos seleccionados para analizar la distribución de beneficios.
def bootstraping(predictions, n_samples=1000):
    
    state = np.random.RandomState(12345)
    profits_list = []

    # Bucle para generar n_samples muestras de bootstrap
    for i in range(n_samples):
        # Toma una muestra aleatoria con reemplazo de los pozos seleccionados
        sub_sample = predictions.sample(frac=1, random_state=state, replace=True)
        
        # Calcula el beneficio de la muestra usando la función profit
        profit_rg, sub_spl_predict = profit(sub_sample)
        profits_list.append(profit_rg)

    # Convierte la lista de beneficios en una Serie para analizar la distribución
    profits_sr = pd.Series(profits_list) 
    return profits_sr

# Aplicación de bootstrapping en las predicciones de cada región.
bootstraped_rg_0 = bootstraping(predict_rg0_df)
bootstraped_rg_1 = bootstraping(predict_rg1_df)
bootstraped_rg_2 = bootstraping(predict_rg2_df)

### Beneficio promedio

In [31]:
print("Región 0: ${:,.2f}".format(bootstraped_rg_0.mean()))
print("Región 1: ${:,.2f}".format(bootstraped_rg_1.mean()))
print("Región 2: ${:,.2f}".format(bootstraped_rg_2.mean()))

Región 0: $38,807,929.57
Región 1: $24,824,230.83
Región 2: $34,966,032.75


En términos de beneficio bruto promedio, la región 0 lidera con `$38,807,929.57`, seguida de cerca por la región 2 con `$34,966,032.75`. En contraste, la región 1 muestra un promedio significativamente menor, con `$24,824,230.83.` Esto implica que, de basarnos en el modelo predictivo, la recomendación inicial sería invertir en la región 0 debido a sus mayores beneficios esperados.

### Intervalo de confianza del 95%.

In [32]:
def intervalo_confianza (bootstraped_prediction):
    # Calcula y almacena percentil del 2.5%
    lower =  bootstraped_prediction.quantile(0.025)
    
    # Calcula y almacena percentil del 97.5%
    upper = bootstraped_prediction.quantile(0.975)
    return lower, upper

lower_0, upper_0 =  intervalo_confianza(bootstraped_rg_0)
lower_1, upper_1 =  intervalo_confianza(bootstraped_rg_1)
lower_2, upper_2 =  intervalo_confianza(bootstraped_rg_2)

print("Intervalo de confianza 95% región 0: \n${:,.2f} \n${:,.2f}\n".format(lower_0, upper_0))
print("Intervalo de confianza 95% región 1: \n${:,.2f} \n${:,.2f}\n".format(lower_1, upper_1))
print("Intervalo de confianza 95% región 2: \n${:,.2f} \n${:,.2f}\n".format(lower_2, upper_2))

Intervalo de confianza 95% región 0: 
$38,116,930.62 
$39,509,833.19

Intervalo de confianza 95% región 1: 
$24,790,020.52 
$24,858,825.78

Intervalo de confianza 95% región 2: 
$34,225,601.96 
$35,734,532.05



Los intervalos de confianza del 95% también respaldan esta conclusión. El intervalo de la región 0 se encuentra entre `$38,116,930.62` y `$39,509,833.19`, y el de la región 2, entre `$34,225,601.96` y `$35,734,532.05`, mientras que el de la región 1 es considerablemente más bajo, con un rango entre `$24,790,020.52` y `$24,858,825.78`. Esto sugiere que los beneficios esperados de las regiones 0 y 2 tienen una variabilidad similar, pero ambas superan a la región 1 con consistencia.


### Riesgo de pérdidas.

In [33]:
# Riesgo de pérdidas
def calcular_riesgo(bootstraped_prediction):
    riesgo = (bootstraped_prediction < 0).mean() * 100
    return riesgo

riesgo_0 = calcular_riesgo(bootstraped_rg_0)
riesgo_1 = calcular_riesgo(bootstraped_rg_1)
riesgo_2 = calcular_riesgo(bootstraped_rg_2)

print("Riesgo de pérdidas región 0: {:.2f}%".format(riesgo_0))
print("Riesgo de pérdidas región 1: {:.2f}%".format(riesgo_1))
print("Riesgo de pérdidas región 2: {:.2f}%".format(riesgo_2))

Riesgo de pérdidas región 0: 0.00%
Riesgo de pérdidas región 1: 0.00%
Riesgo de pérdidas región 2: 0.00%


El análisis de las tres regiones sugiere que invertir en cualquiera de ellas podría resultar rentable, ya que ninguna presenta riesgo de pérdidas (probabilidad de ganancia negativa del 0%). Sin embargo, al comparar el beneficio promedio y los intervalos de confianza de cada región, las regiones 0 y 2 destacan como las opciones más rentables, superando ampliamente a la región 1.

### Conclusión

Al analizar las predicciones versus los datos reales (validacion del modelo), encontramos que el modelo es razonablemente preciso, ya que las ganancias brutas obtenidas caen dentro del intervalo de confianza del 95%. Sin embargo, el margen de error podría reducirse. Dado que el modelo actual presenta un error cuadrático medio (RMSE) que deja margen de mejora, sería recomendable considerar otros modelos o ajustes para mejorar la precisión predictiva.

En cuanto al umbral de producción necesario para justificar la inversión, la región 0 y la región 2 también se destacan, con diferencias respecto al umbral de 43.13 y 38.87 respectivamente, mientras que la región 1 tiene una diferencia de 27.58. Esto refuerza la viabilidad de invertir en las regiones 0 y 2. Si se considera el beneficio real, la región 2 podría tener un rendimiento ligeramente superior en comparación con la región 0, pero las diferencias son marginales y la inversión en cualquiera de las dos sería ventajosa.

### Recomendación Final
Considerando los resultados de este análisis y los anteriores, la región 0 emerge como la opción recomendada para invertir, ya que presenta el mayor beneficio promedio en todas las predicciones y sus valores están respaldados por un intervalo de confianza del 95%, lo cual añade solidez a los resultados. La discrepancia observada entre la rentabilidad de las regiones 0 y 2 en las predicciones versus los datos reales podría estar influida por el índice de RMSE, lo que sugiere cierto margen de error en el modelo predictivo. Sin embargo, dado que el beneficio promedio de la región 0 cae dentro del intervalo de confianza, se considera que la predicción es suficientemente confiable.

La diferencia entre los valores predichos y los datos reales no invalida la recomendación hacia la región 0, ya que su rentabilidad se ha mostrado robusta en cada análisis de las predicciones. En conclusión, aunque la región 2 presenta una rentabilidad atractiva en los datos reales, la consistencia y el respaldo estadístico de los valores predichos inclinan la recomendación final hacia invertir en la región 0.